In [1]:
"""
抓取链家二手房网站中的房源信息
"""

# url=https://nj.lianjia.com/ershoufang/pg{}/
# 每一个房源的链接：   //li[@class="clear LOGVIEWDATA LOGCLICKDATA"]   这里有个小细节class会变，随着滑轮变，所以看好了
#每一个房源的小区名称：.//div[@class="positionInfo"]/a[1]/text() 接着上面的
#每一个房源的小区区域：.//div[@class="positionInfo"]/a[2]/text() 接着上面的
# .//div[@class="houseInfo"]/text()  然后再去切割就行了 3室1厅 | 82.81平米 | 南 | 精装 | 27层 | 2012年建 | 塔楼
# 总价：.//div[@class="totalPrice totalPrice2"]/span/text()
# 单价 ：.//div[@class='unitPrice']/span


'\n抓取链家二手房网站中的房源信息\n'

In [10]:
import requests
from lxml import etree
import random
import time
from fake_useragent import UserAgent
import csv
class LianjiaSpider:
    def __init__(self):
        self.url='https://nj.lianjia.com/ershoufang/pg{}/'
        #这边不写agent了，我们每一页面随机生成一个\

        self.f=open("ershoufang.csv","a",encoding="utf-8",newline="")
        self.writer=csv.writer(self.f)
        #定义一个空列表，大列表
        self.all_list=[]

    def get_html(self,url):
        headers={"Use-Agent":UserAgent().random}
        for i in range(3):  #给三次机会
            try:
                html=requests.get(url=url,headers=headers,timeout=3).text  #给他3秒钟
                self.parse_html(html)
                break
            except Exception as e:
                print("Retry..............")



    def parse_html(self,html):
        p=etree.HTML(html)
        #1、基准xpath
        li_list=p.xpath('//li[@class="clear LOGVIEWDATA LOGCLICKDATA"]')
        item={}
        for li in li_list:
            name_list=li.xpath('.//div[@class="positionInfo"]/a[1]/text()') #小区的名字
            item["name"]=name_list[0].strip() if name_list else None   #防止列表是空，挂掉，如果没取到，那就给它赋值None
            address_list=li.xpath('.//div[@class="positionInfo"]/a[2]/text()')#小区的区域
            item["address"]=address_list[0].strip() if address_list else None
            #3室1厅 | 82.81平米 | 南 | 精装 | 27层 | 2012年建 | 塔楼
            info_li=li.xpath('.//div[@class="houseInfo"]/text()')  #这是一个列表
            if info_li:
                info_li=info_li[0].split("|")  #字符串切割
                if len(info_li) == 7:
                    item["model"]=info_li[0].strip() #规模
                    item["area"] = info_li[1].strip()    
                    item["direct"] = info_li[2].strip()
                    item["perfect"] = info_li[3].strip()
                    item["floor"] = info_li[4].strip()
                    item["year"] = info_li[5].strip()                                
                    item["type"] = info_li[6].strip()
                else:#尽可能精准吧
                    item["model"]=item["area"]=item["direct"]=item["perfect"]=item["floor"]=item["year"]=item["type"]=None
            else:#尽可能精准吧
                item["model"]=item["area"]=item["direct"]=item["perfect"]=item["floor"]=item["year"]=item["type"]=None
            total_list=li.xpath('.//div[@class="totalPrice totalPrice2"]/span/text()')
            item["total"]=total_list[0].strip() if total_list else None
            unit_list=li.xpath('.//div[@class="unitPrice"]/span/text()')
            item["unit"]=total_list[0].strip() if unit_list else None
            print(item)
            list=[item["name"],item["address"],item["model"],item["area"],item["direct"],item["perfect"],item["floor"],item["year"],item["type"],item["total"],item["unit"]]
            self.all_list.append(list)

    def run(self):
        for pg in range(1,100):
            url=self.url.format(pg)
            self.get_html(url)
            time.sleep(random.randint(1,2))
            self.writer.writerows(self.all_list)
        self.f.close()

if __name__ == "__main__":
    spider=LianjiaSpider()
    spider.run()





{'name': '中海桃源里', 'address': '热河南路', 'model': '4室2厅', 'area': '157.21平米', 'direct': '南 北', 'perfect': '毛坯', 'floor': '高楼层(共15层)', 'year': '2018年建', 'type': '板楼', 'total': '950', 'unit': '950'}
{'name': '石榴园', 'address': '新街口', 'model': '3室1厅', 'area': '103.54平米', 'direct': '南', 'perfect': '简装', 'floor': '低楼层(共7层)', 'year': '1996年建', 'type': '板楼', 'total': '395', 'unit': '395'}
{'name': '天泓山庄云山苑', 'address': '马群', 'model': '4室2厅', 'area': '160.1平米', 'direct': '南 北', 'perfect': '精装', 'floor': '中楼层(共11层)', 'year': '2009年建', 'type': '板楼', 'total': '574', 'unit': '574'}
{'name': '华润国际社区', 'address': '顶山街道', 'model': '3室2厅', 'area': '87.93平米', 'direct': '南 北', 'perfect': '精装', 'floor': '高楼层(共33层)', 'year': '2017年建', 'type': '板楼', 'total': '408', 'unit': '408'}
{'name': '天地新城天柱座', 'address': '岔路口', 'model': '3室1厅', 'area': '102.03平米', 'direct': '南 北', 'perfect': '精装', 'floor': '中楼层(共11层)', 'year': '2006年建', 'type': '板塔结合', 'total': '259', 'unit': '259'}
{'name': '珠江路(玄武)', 'address': '珠江路', '